# Day 7

Another conference day, so explanations will come later.

In [1]:
from utils import read_input
from collections import namedtuple

Command = namedtuple('Command', ['command', 'target'], defaults=(None, None))
Directory = namedtuple('Directory', ['name'])
File = namedtuple('File', ['name', 'size'])

def transformer(line):
    if line.startswith('$'):
        prompt = line.split(' ')
        if len(prompt) == 2:
            return Command(command=prompt[1])
        elif len(prompt) == 3:
            return Command(command=prompt[1], target=prompt[2])
    elif line[0].isnumeric():
        size, name = line.split(' ')
        return File(name, int(size))
    elif line.startswith('dir'):
        _, name = line.split(' ')
        return Directory(name)
    else:
        raise Exception('Unknown line')
        
listings = read_input(7, transformer)
examples = read_input(7, transformer, True)

class FileObj:
    
    def __init__(self, name, size, is_dir):
        self.name = name
        self.is_dir = is_dir
        self.size = size
        self.children = []
        
    def add_child(self, child):
        self.children.append(child)
        
    def calculate_size(self):
        s = self.size
        for child in self.children:
            s += child.calculate_size()
        return s
    
    def sum_size(self, threshold):
        if not self.is_dir:
            return 0
        elif self.calculate_size() <= threshold:
            return self.calculate_size() + sum(child.sum_size(threshold) for child in self.children)
        else:
            return sum(child.sum_size(threshold) for child in self.children)
        
    def find_larger_than(self, threshold):
        if not self.is_dir:
            return None
        elif self.calculate_size() >= threshold:
            return [self.calculate_size()] + [child.find_larger_than(threshold) for child in self.children if child.find_larger_than(threshold)]
        else:
            return None
    
    def __repr__(self):
        if self.is_dir:
            return f'<Dir name="{f"/{self.name}"}" children={self.children} size={self.calculate_size()}>'
        else:
            return f'<File name="{self.name}" size={self.size}>'

def process(listings):
    root = FileObj('/', 0, True)
    current = root
    path = [root]
    for listing in listings:
        match listing:
            case Directory(name):
                obj = FileObj(name, 0, True)
                current.add_child(obj)
            case File(name=name, size=size):
                obj = FileObj(name, size, False)
                current.add_child(obj)
            case Command('ls', None):
                continue
            case Command('cd', '..'):
                path.pop()
                current = path[-1]
            case Command('cd', None):
                current = root
                path = [root]
            case Command('cd', directory):
                if(directory == '/'):
                    current = root
                    path = [root]
                else:
                    current = [d for d in current.children if d.name == directory][0]
                    path.append(current)
    return root

In [2]:
root = process(listings.copy())

solution_1 = root.sum_size(100000)
print(f'Part 1: {solution_1}')
assert solution_1 == 1350966

Part 1: 1350966


## Part 2

In [4]:
from utils import flatten

TOTAL_DISK_SPACE = 70000000
AVAILABLE_NEEDED = 30000000

root = process(listings.copy())

current_space = root.calculate_size()
space_needed = AVAILABLE_NEEDED - (TOTAL_DISK_SPACE - current_space)

candidates = flatten(root.find_larger_than(space_needed))

solution_2 = min(candidates)
print('Part 2:', solution_2)
assert solution_2 == 6296435

Part 2: 6296435
